In [ ]:

import subprocess
import os

!source /etc/network_turbo

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value
        
import os
# We need Hugging Face の 镜像网站！
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"


In [ ]:
from datasets import load_dataset
import time
start_time = time.time()
valid_dataset = load_dataset("json", data_files="data/translation2019zh_valid.json")
end_time = time.time()
print(f"加载valid集耗时: {end_time - start_time:.2f} 秒")


valid_dataset

In [ ]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline



model_name = "t5-small"
translator = pipeline("translation", model=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

max_length = 1024
def preprocess_function(self, examples):
    """预处理函数"""
    inputs = [f"translate English to Chinese: {ex}" for ex in examples["english"]]
    targets = [ex for ex in examples["chinese"]]
    
    model_inputs = tokenizer(
        inputs, 
        text_target=targets, 
        max_length=max_length, 
        truncation=True,
        padding=False
    )
    return model_inputs


tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets.column_names,
)

In [ ]:


from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    output_dir="model_dir",
    eval_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True, #change to bf16=True for XPU
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

evaluate_result = trainer.evaluate()
print("Evaluation Results:",evaluate_result)

test_samples = split_datasets["test"][:5]
inputs = [sample["english"] for sample in test_samples]
predictions = translator(inputs)
for i, pred in enumerate(predictions):
    print(f"Sample {i+1}:")
    print(f"Input: {inputs[i]}")
    print(f"Prediction: {pred['translation_text']}")